**실행 후, 추출된 파일 내용 의미**
1. text: HTML코드에서 추출한 문구
2. sentiment_label: 아래 단계 중 하나
3. sentiment_score: 감정의 강도 또는 확신도 (값이 높을수록 해당 감정 레이블에 대한 확신이 크다는 뜻)

<단계>

*   1 star: 매우 부정적인 감정
*   2 stars: 다소 부정적인 감정
*   3 stars: 중립적인 감정
*   4 stars: 긍정적인 감정
*   5 stars: 매우 긍정적인 감정








In [ ]:
# 1. 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 감정 분석 파이프라인 로드 (구글 드라이브 연결할 때마다 실행)
import os
import json
import re
from transformers import pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [19]:
# 파일 경로 설정(각자 알아서)
input_file_path = '/content/drive/MyDrive/test_result(1~250).json'  # 입력 파일 경로
output_file_path = '/content/drive/MyDrive/sentiment_results(1~250).json'  # 출력 파일 경로

In [ ]:
# HTML에서 텍스트 추출 및 감정 분석 함수 정의

def extract_text_from_html(html_code):
    """HTML 코드에서 텍스트를 추출하고 불필요한 공백 제거"""
    # 스크립트, 스타일, 주석 제거
    html_code = re.sub(r'<script.*?>.*?</script>', '', html_code, flags=re.DOTALL)
    html_code = re.sub(r'<style.*?>.*?</style>', '', html_code, flags=re.DOTALL)
    html_code = re.sub(r'<!--.*?-->', '', html_code, flags=re.DOTALL)

    # HTML 태그 제거 및 공백 정리
    text = re.sub('<[^<]+?>', '', html_code)  # HTML 태그 제거
    text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나의 공백으로 치환
    return text.strip()


def split_text(text, max_length=512):
    """텍스트를 max_length 이하의 조각으로 문장이 끊기지 않도록 나누는 함수"""
    # 텍스트를 단어 단위로 분할
    words = text.split()
    chunks = []
    chunk = ""

    for word in words:
        # 현재 조각에 단어를 추가했을 때 max_length를 초과하지 않으면 추가
        if len(chunk) + len(word) + 1 <= max_length:  # +1은 공백 추가를 고려한 것
            chunk += (" " + word if chunk else word)
        else:
            # 현재 조각이 max_length에 가까우면 조각을 추가하고 새로운 조각 시작
            chunks.append(chunk)
            chunk = word

    # 마지막 조각 추가
    if chunk:
        chunks.append(chunk)

    return chunks

def analyze_sentiment(data):
    """JSON 데이터에서 HTML 텍스트를 추출하고 감정 분석을 수행"""
    results = {}
    for key, content in data.items():
        html_code = content.get("HTML", "")
        text = extract_text_from_html(html_code)

        # 감정 분석 수행
        if text:
            text_chunks = split_text(text, max_length=256)
            sentiments = [sentiment_analyzer(chunk)[0] for chunk in text_chunks]

            # 각 조각의 감정 분석 결과를 결합 (예: 'label'을 기준으로 가장 많이 나타난 레이블 선택)
            labels = [sent["label"] for sent in sentiments]
            combined_label = max(set(labels), key=labels.count)

            # 점수는 평균화하여 결합
            scores = [sent["score"] for sent in sentiments if sent["label"] == combined_label]
            average_score = sum(scores) / len(scores)

            results[key] = {
                "text": text,
                "sentiment_label": combined_label,
                "sentiment_score": average_score
            }
    return results

# JSON 파일 로드
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 감정 분석 수행 및 결과 저장
sentiment_results = analyze_sentiment(data)
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(sentiment_results, output_file, ensure_ascii=False, indent=4)

print(f"Sentiment analysis results saved to {output_file_path}")

In [ ]:
import json
import pandas as pd

# JSON 파일 로드
with open(output_file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# 판다스 데이터프레임으로 변환
df = pd.DataFrame(json_data)

# 출력 설정 변경
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)        # 자동으로 너비 조정

# 결과 출력
df.T